<a href="https://colab.research.google.com/github/ShriNarayanPandey01/DataScience-DeepLearning/blob/main/PreProcessingData/MixedCsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Jupyter Notebook tells
**How to handle CSV files containing mixed Data**
type while training Machine Learning model in tensorFlow

This is a jupyter notebook i created while i was Learning Machine Learning
I hope you find this notebook interesting and are able to understand this notebook easily :)

This Jupyter notebook showcase what i learned from https://www.tensorflow.org/tutorials/load_data/csv

In [2]:
import pandas as pd

In [3]:
titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In order to Learn How to handle CSV files with mixed Datatypes we will Take an example of famous  Titanic passenger information dataset


---
As you can see The given dataset contains data in both format String as well as numeric,Our machine learning model does not understand string input it only understand numeric value

---
So in order for machine learning model to train with String data we need to first convert the String data into numeric datatype.

We can do it by Various Method :


1.   We can process the data offline to convert String data into catagorical numeric data type and fetch the new dataset to machine Learning model ,However this method require you  to preprocess the String data all the time before you pass it to train the model or make prediction from it which is not ideal.
2.   We can add a Preprocessing layer into our machine Learning model which will automaticlaly convert all the String data into numerical values and we dont have to preprocess the String data all the time


In [4]:
titanic_ftr = titanic.copy()
titanic_lbl = titanic_ftr.pop('survived')

In [5]:
titanic_ftr.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [6]:
titanic_lbl.head()

0    0
1    1
2    1
3    1
4    0
Name: survived, dtype: int64

**Now we have Seperated our Features and Output**

In [7]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

We would be using Keras Functional API to implement our preprocessing logic

Using tf.keras.Input we would create a layer that would take input of specific type and shape

In [8]:
# Create a symbolic input
#shape=() defines the shape of input to be given
#dtype define the dataType of Input
input = tf.keras.Input(shape=(), dtype=tf.float32)

#result Define the Set of operation that need to be performed on given Input
# Perform a calculation using the input
result = 2*input + 1

# the result doesn't have a value
result

<KerasTensor: shape=(None,) dtype=float32 (created by layer 'tf.__operators__.add')>

In [9]:
# we are creating a layer the would perform a predifined set of operation on input Like converting String into numeric dataType
# input define the input dataType and output defines operation to be performed input data
demo = tf.keras.Model(inputs=input, outputs=result)
print(demo(np.array([1])).numpy())

[3.]


To build the preprocessing model, start by building a set of symbolic tf.keras.Input objects for each Feature in dataset, matching the names and data-types of the CSV columns.

In [13]:
inputs = {}
for name, column in titanic_ftr.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

{'sex': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'sex')>,
 'age': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'age')>,
 'n_siblings_spouses': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'n_siblings_spouses')>,
 'parch': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'parch')>,
 'fare': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fare')>,
 'class': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'class')>,
 'deck': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'deck')>,
 'embark_town': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'embark_town')>,
 'alone': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'alone')>}

Now we have created a tf.keras.Input Symbolic for each feature in our dataset

This below code define the Operation that should be performed on numeric datatype.
We are Normalization on our numeric data if you want to know why we are normalizing our datatype please refer to this link---
https://www.almabetter.com/bytes/tutorials/data-science/normalization-in-machine-learning

In [19]:
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(titanic[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)
# print(x)
all_numeric_inputs

<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'normalization_5')>

Now we add the Operation that are to be performed on Numeric data type into a list

In [21]:
preprocessed_inputs = [all_numeric_inputs]

KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name=None), name='normalization_5/truediv:0', description="created by layer 'normalization_5'")


The code below showcases the operations that are to be performed on String data


---

In order to convert String into numeric data Type we are using


1.   tf.keras.layers.StringLookup which maps each unique string to Integer indices in vocabulary eg. Apple is mapped to 1 so whenever we wanna write apple we would write its respective index in vocabulary
2.   tf.keras.layers.CategoryEncoding which converts indices into tf.float32 data which appropriate for training



In [22]:
for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  lookup = layers.StringLookup(vocabulary=np.unique(titanic_ftr[name]))
  one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)
preprocessed_inputs

[<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'normalization_5')>,
 <KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'category_encoding')>,
 <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'category_encoding_1')>,
 <KerasTensor: shape=(None, 9) dtype=float32 (created by layer 'category_encoding_2')>,
 <KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'category_encoding_3')>,
 <KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'category_encoding_4')>]

Now we Combine all operation we have defined for String and numeric data type
and create a preprocessing layer for our model

In [23]:
preprocessed = tf.keras.layers.Concatenate()(preprocessed_inputs)
preprocessing = tf.keras.Model(inputs, preprocessed)

In [24]:
titanic_features_dict = {name: np.array(value)
                         for name, value in titanic_ftr.items()}
features_dict = {name:values[:1] for name, values in titanic_features_dict.items()}
preprocessing(features_dict)

<tf.Tensor: shape=(1, 28), dtype=float32, numpy=
array([[-0.610415 ,  0.395198 , -0.4790527, -0.4974028,  0.       ,
         0.       ,  1.       ,  0.       ,  0.       ,  0.       ,
         1.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  1.       ,
         0.       ,  0.       ,  0.       ,  1.       ,  0.       ,
         0.       ,  1.       ,  0.       ]], dtype=float32)>

**Now we build our machine Learning model for training**

In [30]:
model = tf.keras.Sequential([
    layers.Dense(64,activation='relu'),
    layers.Dense(1)
])
processing_inputs = preprocessing(inputs)
result = model(processing_inputs)
Model = tf.keras.Model(inputs,result)
Model.compile(loss=tf.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam')
Model.fit(titanic_features_dict,titanic_lbl,epochs=10)


Epoch 1/10
20/20 [==============================] - 1s 5ms/step - loss: 0.6092
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5392
Epoch 3/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5020
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4733
Epoch 5/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4543
Epoch 6/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4400
Epoch 7/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4300
Epoch 8/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4241
Epoch 9/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4178
Epoch 10/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4167


In [32]:
model2 = tf.keras.Sequential([
    layers.Dense(64,activation='relu'),
    layers.Dense(1)
])
model2 = tf.keras.Model(inputs,model2(processing_inputs))
model2.compile(loss=tf.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam')
model2.fit(titanic_features_dict,titanic_lbl,epochs=10)

Epoch 1/10
20/20 [==============================] - 1s 5ms/step - loss: 0.7012
Epoch 2/10
20/20 [==============================] - 0s 5ms/step - loss: 0.6039
Epoch 3/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5481
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5089
Epoch 5/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4794
Epoch 6/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4573
Epoch 7/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4417
Epoch 8/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4302
Epoch 9/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4234
Epoch 10/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4170
